In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/Kannada-MNIST/train.csv', index_col=False)
test = pd.read_csv('../input/Kannada-MNIST/test.csv', index_col=False)
validate = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv', index_col = False)

In [ ]:
# copy the 3 datasets and separate X, y
# Turn df into array and scale the X_data between 0 and 1
X_train = train.copy().drop('label', axis = 1).values
X_train = X_train.astype('float32') / 255.0
y_train = train.label

X_test = test.copy().drop('id', axis = 1).values
X_test = X_test.astype('float32')/255.0

X_valid = validate.copy().drop('label', axis = 1).values
X_valid = X_valid.astype('float32')/255.0
y_valid = validate.label

print('X_train data:{}'.format(X_train.shape))
print('y_train data:{}'.format(y_train.shape))
print('X_test data:{}'.format(X_test.shape))
print('X_valid data:{}'.format(X_valid.shape))
print('y_valid data:{}'.format(y_valid.shape))

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 800)
from datetime import datetime
import os

# Convert all object features to category Features
import pandas.api.types as ptypes

# Metrics for classification
from sklearn.metrics import accuracy_score

# Tensoreflow and keras packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten, Reshape, Input,\
                                    Conv2D, MaxPooling2D, Dropout, UpSampling2D, ActivityRegularization

from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import RandomNormal, Constant
from tensorflow.keras.callbacks import EarlyStopping

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

import warnings
warnings.filterwarnings('ignore')

#To display the plots
%matplotlib inline

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
# Model (2) - Denoising Autoencoder
# Create an encoder
keras.backend.clear_session()
start = datetime.now()
tf.random.set_seed(42)
np.random.seed(42)

input_layer = Input(shape = (784,))
# Building Encoder
e1=Dropout(0.5)(input_layer)
e2=Dense(100, activation = 'selu')(e1)
e3=Dense(30, activation = 'selu')(e2)

# Building Decoder and Output
d1 = Dense(30, activation = 'selu')(e3)
d2 = Dense(100, activation = 'selu')(d1)
output = Dense(784,activation="sigmoid")(d2)

denoise_ae = Model(inputs = input_layer, outputs = output)
#optimizer = Adam(learning_rate = 0.0025)
denoise_ae.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics = [rounded_accuracy])
history2 = denoise_ae.fit(X_train, X_train, epochs=20,
                         validation_data=(X_valid, X_valid))
end = datetime.now()
runtime_2 = end - start
print('Denoising Autoencoder Runtime:', runtime_2)

In [ ]:
# Save and reload the encoder for evaluation and TSNE
denoising_encoder = Model(inputs = input_layer, outputs = e3)
# save the encoder to file
denoising_encoder.save('denoising_encoder.h5')
# Load the saved encoder back
denoising_encoder = load_model('denoising_encoder.h5')

In [ ]:
# Using encoder to extract 30 important features
X_train_encoded2 = denoising_encoder.predict(X_train)
X_test_encoded2 = denoising_encoder.predict(X_test)
X_valid_encoded2 = denoising_encoder.predict(X_valid)
print(X_test_encoded2.shape)

In [ ]:
# Using Base Model encoder's output as feature extraction and dimensionality reduction input
# for XGB Classifier to do prediction
start = datetime.now()
import xgboost as xgb
from xgboost import XGBClassifier
model2 = XGBClassifier(eval_metric='mlogloss') 
model2.fit(X_train_encoded2, y_train)
y_pred2 = model2.predict(X_test_encoded2)
end = datetime.now()
runtime = end - start
print(runtime)

In [ ]:
# Predict test data
print(y_pred2.shape)
print(y_pred2[:50])

In [ ]:
sample_submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission = sample_submission.copy()
submission['label']=y_pred2
submission.to_csv('submission.csv', index=False)